# 08 안전 운전자 예측
## 8.3 베이스라인 모델

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = '../../data/08_safe_driver/'

train_df = pd.read_csv(data_path+'train.csv', index_col='id')
test_df = pd.read_csv(data_path+'test.csv', index_col='id')
submission_df = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

### 8.3.1 피처 엔지니어링

In [3]:
# 데이터 합치기
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data.drop(columns='target')

In [4]:
all_features = all_data.columns
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [5]:
# 명목형 피처 원핫 인코딩
from sklearn.preprocessing import OneHotEncoder
cat_features = [feature for feature in all_features if 'cat' in feature]
onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])
encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [6]:
# 필요 없는 피처 제거
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature and feature not in drop_features)]
remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15']

In [7]:
# csr matrix 결합
from scipy import sparse

all_data_sprs = sparse.hstack([
    sparse.csr_matrix(all_data[remaining_features]),
    encoded_cat_matrix,
], format='csr')

In [9]:
# 데이터 나누기
num_train = len(train_df)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]
y = train_df['target'].values

### 8.3.2 평가지표 계산 함수 작성
* 지니 계수란?
    * 경제학에서 소득 불평등 정도를 나타내는 지표
    * 지니계수가 작을 수록 소득 수준이 평등, 클수록 불평등함
    * 로렌츠 곡선을 이용해 계산
        * 인구 누적 비율과 해당 소득 누적 점유율을 연결한 선
    * ![지니 계수](../images/gini계수.PNG)
        * 지니계수 = $\frac{A}{A+B}$ = (2 * ROC AUC - 1)
* 정규화 지니계수 계산 함수
    * 정규화 지니 계수 = $\frac{예측 값에 대한 지니 계수}{예측이 완벽할 때의 지니 계수}$

In [10]:
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    L_mid = np.linspace(1/n_samples, 1, n_samples) # 대각선 값

    # 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()]
    # 로렌츠 곡선
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # array / scalar = array
    G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니계수

    # 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때의 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [11]:
# LightGBM용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label() # 데이터셋의 타깃값을 반환
    #    평가지표 이름       평가 점수     평가 점수가 높을 수록 좋은지 여부
    return 'gini',   eval_gini(labels, preds),      True

### 8.3.3 모델 훈련 및 성능 검증
#### OOF 예측 방식
* OOF 예측(Out of Fold prediction)이란?
    * K 폴드 교차 검증을 수행하면서 각 폴드마다 테스트 데이터로 예측하는 방식
    * 예측 절차
        1. 전체 훈련 데이터를 K개 그룹으로 나눔
        2. K개 그룹 중 한 그룹은 검증 데이터, 나머지 K-1개 그룹은 훈련 데이터로 지정
        3. 훈련 데이터로 모델 훈련
        4. 훈련된 모델을 이용해 검증 데이터로 타깃 확률을 예측하고 전체 테스트 데이터로도 타깃 확률 예측
        5. 검증 데이터로 구한 예측 확률과 테스트 데이터로 구한 예측 확률을 기록
        6. 검증 데이터를 다른 그룹으로 바꿔가며 2~5번 절차를 K번 방복
        7. K개 그룹의 검증 데이터로 예측한 확률을 훈련 데이터 실제 타깃값과 비교해 성능 평가점수 계산
        8. 테스트 데이터로 구한 K개 예측 확률의 평균을 구함
    * 서로 다른 K개 모델을 만듬
    * 예측 확률은 K개의 평균으로 도출
    * 장점
        * 과대적합 방지 효과
        * 앙상블 효과

In [12]:
# OOF 방식으로 LightGBM 훈련
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [13]:
params = {
    'objective': 'binary', # 이진분류 문제
    'learning_rate': 0.01,
    'force_row_wise': True, # 경고문구를 없애는 파라미터
    'random_state': 0,
}

In [14]:
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

In [15]:
import lightgbm as lgb

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    lgb_model = lgb.train(
        params=params,             # 훈련용 하이퍼파라미터
        train_set=dtrain,          # 훈련 데이터셋
        num_boost_round=1000,      # 부스팅 반복 횟수
        valid_sets=dvalid,         # 상능 평가용 검증 데이터셋
        feval=gini,                # 검증용 평가지표
        early_stopping_rounds=100, # 조기종료 조건
        verbose_eval=100           # 100번째마다 점수 출력
    )
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\dukim\miniconda3\envs\musthave_mldl_problem_solving_strategy\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.153354	valid_0's gini: 0.261651
[200]	valid_0's binary_logloss: 0.152426	valid_0's gini: 0.275704
[300]	valid_0's binary_logloss: 0.152023	valid_0's gini: 0.282277
[400]	valid_0's binary_logloss: 0.1518	valid_0's gini: 0.286648
[500]	valid_0's binary_logloss: 0.151713	valid_0's gini: 0.287944
[600]	valid_0's binary_logloss: 0.151672	valid_0's gini: 0.288637
[700]	valid_0's binary_logloss: 0.151659	valid_0's gini: 0.288939
Early stopping, best iteration is:
[681]	valid_0's binary_logloss: 0.151659	valid_0's gini: 0.289034
폴드 1 지니계수 : 0.2890336154188232

##########

In [16]:
print(f'OOF 검증 데이터 지니계수: {eval_gini(y, oof_val_preds)}')

OOF 검증 데이터 지니계수: 0.2804995714877777


### 8.3.4 예측 및 결과 제출

In [18]:
submission_df['target'] = oof_test_preds
submission_df.to_csv(data_path+'submission_base.csv')